In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stanford-plato-corpus/data_per_paragraph.csv
/kaggle/input/stanford-plato-corpus/corpus_articles.csv
/kaggle/input/stanford-plato-corpus/1-make-data-csv.ipynb
/kaggle/input/stanford-plato-corpus/2-explode-nested-data.ipynb
/kaggle/input/stanford-plato-corpus/make-final-plato-corpus.ipynb
/kaggle/input/stanford-plato-corpus/plato_mirror_spr2020.tgz
/kaggle/input/stanford-plato-corpus/simple_data.csv
/kaggle/input/stanford-plato-corpus/data.csv
/kaggle/input/stanford-plato-corpus/get-data-for-title.ipynb
/kaggle/input/stanford-plato-corpus/corpus_article_text_features.csv


In [2]:
# df.head()

In [3]:

import re
def simple_preprocess(text):
    text = text.replace('\n', ' ') #Remove line breaks
    matches = re.findall(r'\(([^()]*)\)', text)  #Remove citations
    for i in matches:
        if re.search(r'[0-9][0-9][0-9][0-9]?', i):
            text = text.replace('({})'.format(i), '')
        elif len(i) < 20:
            text = text.replace('({})'.format(i), '')
    return text


#usage get_data(title)
#returns list of dictionaries, preamble
#where each dictionary has only one key and value, key is a tuple like (section_id, paragraph_id) and value is the paragraph_text. 

def get_data(title):
    df = pd.read_csv('/kaggle/input/stanford-plato-corpus/data_per_paragraph.csv')    
    df['preamble_text'] = df['preamble_text'].fillna(str(''))
    df['section.paragraph.text'] = df['section.paragraph.text'].fillna(str(''))
    if title in list(df['title']):
        req_df = df.loc[df['title'] == title]
        req_df['section.processed_paragraph.text'] = req_df['section.paragraph.text'][0:5].apply(simple_preprocess)
        data = [{(s,p_id):p} 
                            for s,p_id,p in zip(req_df['section.id'], 
                                                req_df['section.paragraph.id'],
                                                req_df['section.processed_paragraph.text'])]
        preamble = req_df['preamble_text'][1]
        preamble = simple_preprocess(preamble)
        data.append({'preamble_text':preamble})
#         print(req_df['preamble_text'][1])
        return data, preamble
    else:
        print('title not found in the plato corpus')
        return None

In [4]:
title = 'Medieval Theories of the Categories'

In [5]:
data, preamble = get_data(title)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
type(data)

list

In [7]:

preamble




'   This entry is intended as a brief and general introduction to the development of category theory from the beginning of the Middle Ages, in the sixth century, to the Silver Age of Scholasticism, in the sixteenth. This development is fascinating but extraordinarily complex.  Scholars are just beginning to take note of the major differences in the understanding of categories and of how these differences are related to the discussion of other major philosophical topics in the Middle Ages. Much work remains to be done, even regarding the views of towering figures, so necessarily we have had to restrict our discussion to only a few major figures and topics. Still, we hope that the discussion will serve as a good starting point for anyone interested in category theory and its history. '

In [8]:
! wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-05-26 01:18:16--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-05-26 01:18:16--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-05-26 01:18:17--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [9]:
! unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [10]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

def remove_stopwords(sen):
  sen_new = " ".join([i for i in sen if i not in stopwords.words('english')])
  return sen_new

f=[]
f_tag=[]
print(type(data))
for k in data:
    for i in k.keys():
        if len(str(k[i]))>5:
            d=sent_tokenize(k[i])
            for j in d:
                j_clean = j.replace("[^a-zA-Z]", " ").lower()
                j_clean1= remove_stopwords(j_clean.split())
                f=f+[str(j_clean1)]
                f_tag=f_tag+['#'+str(i[0])+','+str(i[1])+'#'+str(j_clean1)]
               
print(f,f_tag)
# Extract word vectors
word_embeddings = {}
g = open('glove.6B.100d.txt', encoding='utf-8')
for line in g:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
g.close()
sentence_vectors = []
for i in f:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

    

<class 'list'>
['philosophers speak categories many different ways.', 'one initial, rather substantial, difference philosophers allow large number categories allow small number.', 'first include among categories different things human, green, animal, thought, justice; second speak general things substance, quality, relation, like, categories.', 'among twentieth-century authors allow many categories gilbert ryle .', 'roderick chisholm example few.', "medieval authors follow aristotle's narrow understanding.", 'disagreement concerning categories history philosophy end there.', 'even restrict discussion small number items sort aristotle regards categories, many issues remain settled them, philosophers frequently disagree settle them.', 'issues may gathered roughly ten groups.', 'first group comprises may described roughly extensional issues; number categories.', 'extension term comprised things term truthfully predicated.', 'thus extension ‘cat’ consists animals true say cats.', 'philosop

In [11]:
sim_mat = np.zeros([len(f), len(f)])

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
for i in range(len(f)):
  for j in range(len(f)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [14]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [15]:
ranked_sentences_tag = sorted(((scores[i],s) for i,s in enumerate(f_tag)), reverse=True)
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(f)), reverse=True)

In [16]:
sn = 10

# Generate summary
for i in range(sn):
  print(ranked_sentences[i][1])

first include among categories different things human, green, animal, thought, justice; second speak general things substance, quality, relation, like, categories.
three common questions raised context are: categories extra-mental entities, properties, qualities, relations, structures, sets, classes, forms?
one initial, rather substantial, difference philosophers allow large number categories allow small number.
third group ontological; issues involve place categories occupy map things exist exist.
scholars beginning take note major differences understanding categories differences related discussion major philosophical topics middle ages.
much work remains done, even regarding views towering figures, necessarily restrict discussion major figures topics.
even restrict discussion small number items sort aristotle regards categories, many issues remain settled them, philosophers frequently disagree settle them.
categories also thought extra-mental features things think speak have–call pro

In [17]:
pip install text2text

     |████████████████████████████████| 46 kB 995 kB/s 
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [18]:
from text2text.text_generator import TextGenerator
qg = TextGenerator(output_type="question")

qg.predict(ranked_sentences[i][1])

Better speed can be achieved with apex installed from https://www.github.com/artitw/apex.


100%|██████████| 213450/213450 [00:00<00:00, 393816.03B/s]


***** Recover model: %s ***** qg_model.bin


100%|██████████| 1242874899/1242874899 [01:47<00:00, 11527177.20B/s]


IndexError: list index out of range